Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StanModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path_s("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchains);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
parameters        = alpha, beta, sigma

Empirical Posterior Estimates
──────────────────────────────────────────────────
parameters
        Mean     SD   Naive SE  MCSE   ESS
alpha 154.6021 0.2727   0.0043 0.0053 1000
 beta   0.9051 0.0421   0.0007 0.0006 1000
sigma   5.1036 0.1943   0.0031 0.0033 1000

Quantiles
──────────────────────────────────────────────────
parameters
        2.5%     25.0%    50.0%    75.0%    97.5% 
alpha 153.6030 154.4200 154.6020 154.7850 155.5800
 beta   0.7639   0.8766   0.9053   0.9336   1.0522
sigma   4.4191   4.9685   5.0982   5.2317   6.0142



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type Chains, with data of type 1000×3×4 Array{Float64,3}

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
internals         = 
parameters        = alpha, beta, sigma

parameters
        Mean     SD   Naive SE  MCSE   ESS
alpha 154.6021 0.2727   0.0043 0.0053 1000
 beta   0.9051 0.0421   0.0007 0.0006 1000
sigma   5.1036 0.1943   0.0031 0.0033 1000



Should be identical to earlier result

In [9]:
describe(chn2)

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
parameters        = alpha, beta, sigma

Empirical Posterior Estimates
──────────────────────────────────────────────────
parameters
        Mean     SD   Naive SE  MCSE   ESS
alpha 154.6021 0.2727   0.0043 0.0053 1000
 beta   0.9051 0.0421   0.0007 0.0006 1000
sigma   5.1036 0.1943   0.0031 0.0033 1000

Quantiles
──────────────────────────────────────────────────
parameters
        2.5%     25.0%    50.0%    75.0%    97.5% 
alpha 153.6030 154.4200 154.6020 154.7850 155.5800
 beta   0.7639   0.8766   0.9053   0.9336   1.0522
sigma   4.4191   4.9685   5.0982   5.2317   6.0142



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*